In [58]:
# Importar las librerías necesarias
%pip install agentpy pathfinding owlready2

In [59]:
import agentpy as ap
import pathfinding as pf
import matplotlib.pyplot as plt
from owlready2 import *
import itertools
import random
import IPython
import math

In [60]:
# Crear y cargar la ontología
onto = get_ontology("file://onto.owl")
onto.destroy(update_relation=True, update_is_a=True)

In [61]:
with onto:
    class Entity(Thing):
        pass

    class Camera(Entity):
        pass

    class StoreObject(Entity):
        pass

    class SecurityGuard(Entity):
        pass

    class Drone(Entity):
        pass

    class Place(Thing):
        pass

    class is_in_place(ObjectProperty):
        domain = [Entity]
        range = [Place]
        pass

    class has_position(ObjectProperty, FunctionalProperty):
        domain = [Entity]
        range = [str]
        pass

    class object_in_store(ObjectProperty):
        domain = [SecurityGuard]
        range = [int]
        pass

    class object_within_reach(ObjectProperty):
        domain = [Drone]
        range = [int]
        pass


In [62]:
# Definir la clase de agentes para las cámaras
class cameraAgent(ap.Agent):
    def setup(self):
        self.agentType = 0  # Tipo de agente para diferenciación

    def see(self, e):
        seeRange = (self.model.p.storeSize[0] // 2) - 1
        objects = [a for a in e.neighbors(self, distance=seeRange) if a.agentType == 3]
        objects_info = [{
            "id": obj.id,
            "type": obj.object_is,
            "position": self.model.Store.positions[obj]
        } for obj in objects]

        return objects_info

    def step(self):
        # Obtener la información de los objetos cercanos
        objects_seen = self.see(self.model.Store)
        if objects_seen:
            # Enviar mensaje al dron para inspeccionar
            self.model.drone[0].receive_message(objects_seen)


In [63]:
# Definir la clase de agentes para los objetos en la tienda
class objectAgent(ap.Agent):
    def setup(self):
        self.agentType = 3  # Tipo de agente para diferenciación
        possible_objects = ["box", "person", "bottle"]
        self.object_is = random.choice(possible_objects)

    def step(self):
        pass  # No se requiere lógica de movimiento para objetos estáticos


In [64]:
# Definir la clase de agentes para los guardias de seguridad
class securityGuardAgent(ap.Agent):
    def setup(self):
        self.agentType = 1  # Tipo de agente para diferenciación
        self.object_in_store = []

    def step(self):
        # Lógica de patrullaje o respuesta a incidentes
        if self.model.messages:
            for message in self.model.messages:
                self.respond_to_incident(message)

    def respond_to_incident(self, message):
        # Responder a un incidente informado por el dron
        print(f"Security guard responding to: {message}")
        # Aquí podrías incluir lógica para mover al guardia al lugar del incidente



In [65]:
# Definir la clase de agentes para los drones
class droneAgent(ap.Agent):
    def setup(self):
        self.agentType = 2  # Tipo de agente para diferenciación
        self.firstStep = True
        self.currentPlan = []
        self.messages = []  # Almacenar mensajes de las cámaras

    def step(self):
        if self.firstStep:
            self.firstStep = False
            # Despegar y empezar patrullaje
            self.take_off()
        elif self.messages:
            # Inspeccionar la zona donde se detectó un objeto sospechoso
            self.inspect_area(self.messages.pop(0))

    def take_off(self):
        print("Drone taking off and starting patrol.")
        # Lógica para mover el dron al aire

    def receive_message(self, objects_info):
        self.messages.append(objects_info)
        print(f"Drone received message: {objects_info}")

    def inspect_area(self, objects_info):
        # Moverse a la ubicación del objeto sospechoso
        target_position = objects_info[0]['position']
        self.move_towards(target_position)
        print(f"Drone inspecting area at {target_position}")

    def move_towards(self, position):
        # Mover el dron hacia la posición objetivo
        current_position = self.model.Store.positions[self]
        path = self.calculate_path(current_position, position)
        for step in path:
            self.model.Store.move_by(self, step)

    def calculate_path(self, start, end):
        # Implementar lógica de pathfinding para mover el dron
        path = []  # Aquí agregarías la lógica para calcular el camino
        return path


In [66]:
# Definir la clase del modelo de la tienda
class StoreModel(ap.Model):
    def setup(self):
        self.messages = []

        self.objects = ap.AgentList(self, self.p.objects, objectAgent)
        self.cameras = ap.AgentList(self, self.p.cameras, cameraAgent)
        self.securityGuards = ap.AgentList(self, self.p.securityGuards, securityGuardAgent)
        self.drone = ap.AgentList(self, self.p.drone, droneAgent)

        self.Store = ap.Grid(self, self.p.storeSize, track_empty=True)

        # Posiciones predefinidas para las cámaras y el dron
        camera_positions = [
            (0, 0),
            (self.p.storeSize[0] - 1, 0),
            (0, self.p.storeSize[1] - 1),
            (self.p.storeSize[0] - 1, self.p.storeSize[1] - 1)
        ]

        drone_position = [(self.p.storeSize[0] // 2, self.p.storeSize[1] // 2)]

        # Asegurar que solo haya tantas cámaras como posiciones definidas
        while len(self.cameras) > len(camera_positions):
            self.cameras.remove(random.choice(self.cameras))

        while len(self.drone) > len(drone_position):
            self.drone.remove(random.choice(self.drone))

        # Añadir agentes a la cuadrícula
        self.Store.add_agents(self.objects, random=True, empty=True)
        self.Store.add_agents(self.cameras, camera_positions, empty=True)
        self.Store.add_agents(self.drone, drone_position, empty=True)

    def step(self):
        self.objects.step()
        self.cameras.step()
        self.securityGuards.step()
        self.drone.step()

    def update(self):
        pass

    def end(self):
        pass


In [67]:
# Función para animar la simulación
def animation_plot(model, ax):
    agent_type_grid = model.Store.attr_grid('agentType')
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Simulación de Almacén \n Time-step: {model.t}")


In [68]:
# Parámetros de la simulación
parameters = {
    "cameras": 4,  # Número de cámaras
    "objects": 10,  # Número de objetos
    "drone": 1,  # Número de drones
    "securityGuards": 1,  # Número de guardias de seguridad
    "storeSize": (15, 15),  # Tamaño de la cuadrícula
    "steps": 2,  # Número de pasos de la simulación
    "seed": 13 * random.random()  # Semilla para la aleatoriedad
}

# Crear figura para la animación
fig, ax = plt.subplots()

# Crear modelo
model = StoreModel(parameters)

# Ejecutar la simulación con animación
animation = ap.animate(model, fig, ax, animation_plot)

# Mostrar la animación
IPython.display.HTML(animation.to_jshtml())